In [ ]:
from yattag import Doc, indent
from yattag.doc import SimpleDoc
from functools import partial
from fastcore.utils import *
from IPython import display

In [ ]:
@patch
def __repr__(self:Doc): return indent(self.getvalue())

@patch
def _repr_markdown_(self:Doc): return f'```html\n{self}\n```'

@patch
def doctype(self:Doc): self.asis('<!DOCTYPE html>')

@patch
def do(self:Doc, tn, *c, **kw): return partial(self.line if c else self.tag, *c, **kw)

@patch
def __getattr__(self:Doc, tn):
    if tn[0]=='_': raise AttributeError
    return partial(self.tag, tn)

In [ ]:
d,t,tx = Doc().tagtext()

d.doctype()
with d.html():
    with d.body():
        with d.h1('Hello, World!', klass='fdsa'):...
d

```html
<!DOCTYPE html>
<html>
  <body>
    <h1 Hello, World! class="fdsa"></h1>
  </body>
</html>
```

In [ ]:
import types, xml.etree.ElementTree as ET
from dataclasses import dataclass
from functools import partial
from litestar.exceptions import NotFoundException
from fastcore.utils import *

In [ ]:
def _attrmap(o):
    return dict(cls='class', klass='class', fr='for').get(o, o)

def xt(tag:str, *c, **kw):
    if len(c)==1 and isinstance(c[0], types.GeneratorType): c = tuple(c[0])
    elif len(c)==1 and isinstance(c[0],str): c = c[0]
    kw = {_attrmap(k.lstrip('_')):str(v) for k,v in kw.items()}
    return [tag.lower(),c,kw]

In [ ]:
g = globals()
tags = '''html head title meta link style body div span p h1 h2 h3 h4 h5 h6 strong em b i u s
strike sub sup hr br img a link nav ul ol li dl dt dd table thead tbody tfoot tr th td caption
col colgroup form input textarea button select option label fieldset legend details summary main
header footer section article aside figure figcaption mark small iframe object embed param video
audio source canvas svg math script noscript template slot'.split()'''.split()
for o in tags: g[o.capitalize()] = partial(xt, o)

def to_xml(node:tuple):
    "Convert `node` to an XML string."
    def mk_el(tag, cs, attrs):
        el = ET.Element(tag, attrib=attrs)
        if isinstance(cs, tuple): el.extend([mk_el(*o) for o in cs])
        elif cs is not None: el.text = str(cs)
        return el

    root = mk_el(*node)
    ET.indent(root)
    return ET.tostring(root, encoding='unicode')

In [ ]:
@dataclass
class TodoItem:
    id: int
    title: str
    done: bool
    def tag(self): return A(self.title, href=f'/todos/{self.id}')

In [ ]:
TODO_LIST: list[TodoItem] = [
    TodoItem(0, title="Start writing TODO list", done=True),
    TodoItem(1, title="???", done=False),
    TodoItem(2, title="Profit", done=False),
]

In [ ]:
def get_todo_by_title(todo_name) -> TodoItem:
    try: return next(o for o in TODO_LIST if o.title == todo_name)
    except: raise NotFoundException(detail=f"TODO {todo_name!r} not found") from None

def add_item(data: TodoItem) -> list[TodoItem]:
    TODO_LIST.append(data)
    return TODO_LIST

def update_item(item_title: str, data: TodoItem) -> list[TodoItem]:
    todo_item = get_todo_by_title(item_title)
    todo_item.title = data.title
    todo_item.done = data.done
    return TODO_LIST

async def get_list(done: bool|None = None) -> str:
    todos = TODO_LIST if done is None else [
        item for item in TODO_LIST if item.done == done]
    tlist = Ul(Li(item.tag()) for item in todos)
    res = Html(Body(H1('TODO list'), tlist))
    return to_xml(res)

In [ ]:
print(await get_list())

<html>
  <body>
    <h1>TODO list</h1>
    <ul>
      <li>
        <a href="/todos/0">Start writing TODO list</a>
      </li>
      <li>
        <a href="/todos/1">???</a>
      </li>
      <li>
        <a href="/todos/2">Profit</a>
      </li>
    </ul>
  </body>
</html>


In [ ]:
print(get_todo_by_title('Profit'))

TodoItem(id=2, title='Profit', done=False)
